In [12]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Lambda, BatchNormalization, Input, Conv1D, TimeDistributed, Flatten, Activation, LSTM, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping, TensorBoard, History, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as KR
import numpy as np
import copy
import time
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam




 ## --- COMMUNICATION PARAMETERS ---

In [13]:


blocklength=128
k = 1 # Bits per Symbol
n = 2 # Channel Use
R = k/n
k_mod = 4  # bits per symbol / channel use
print("L= ",blocklength*k_mod/n)
L = int(blocklength*k_mod/n) # K : Information bits
train_Eb_dB = 0 # Eb/N0 used for training
noise_sigma = np.sqrt(1 / (2  * k_mod * R * 10 ** (train_Eb_dB / 10))) # Noise Standard Deviation

batch_size = 500 # Number of messages used for training, each size = k*L
nb_train_word = 100000




L=  256.0


## --- DATA GENERATION ---

In [14]:
# Generate training binary Data
train_data = np.random.randint(low=0, high=2, size=(nb_train_word, L,k))


In [15]:
train_data.shape

(100000, 256, 1)

## --- MODEL HYPERPARAMETERS ---


In [17]:
early_stopping_patience = 100
epochs = 100
optimizer = Adam(lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=early_stopping_patience)
# Learning Rate Control
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,
                              patience=5, min_lr=0.0001)
# Save the best results based on Training Set
modelcheckpoint = ModelCheckpoint(filepath='./' + 'LSTM' + str(k) + '_' + str(L) + '_' + str(n) + '_' + str(train_Eb_dB) + 'dB' + ' ' + 'AWGN' + '.h5',
                                  monitor='loss',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=True,
                                  mode='auto', period=1)



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## ---DEFINING FUNCTIONS ---

In [18]:
# Define Power Norm for Tx
def normalization(x):
    mean = KR.mean(x ** 2)
    return x / KR.sqrt(2 * mean)  # 2 = I and Q channels

# Define Channel Layers including AWGN and Flat Rayleigh fading
#  x: input data
#  sigma: noise std
def channel_layer(x, sigma):
    w = KR.random_normal(KR.shape(x), mean=0.0, stddev=sigma)
    return x + w


## ---MODEL---

In [ ]:
model_input = Input(batch_shape=(batch_size, L*k,1), name='input_bits')


#Encoding
e = Conv1D(filters=128, strides=1, kernel_size=1 ,padding='same', name='e_0',)(model_input)
e = BatchNormalization()(e)
e = Activation('relu')(e)

e = LSTM(units=128,return_sequences='True',name='e_1')(e)
e = BatchNormalization()(e)
e = Activation('relu')(e)

e = TimeDistributed(Dense(n))(e)
e = Activation('linear')(e)
e = Reshape((int(L*n/k_mod),k_mod))(e)

#Modulation

e = Conv1D(filters=128, strides=1, kernel_size=1, padding="same",)(e)
e = BatchNormalization(name='e_2')(e)
e = Activation('relu', name='e_3')(e)

e = LSTM(units=64,return_sequences='True',name='e_4')(e)
e = BatchNormalization(name='e_5')(e)
e = Activation('tanh', name='e_6')(e)


e = TimeDistributed(Dense(2),name='e_7')(e)
e = Activation('linear', name='e_8')(e)

e = Lambda(normalization, name='power_norm')(e)

# AWGN channel
y_h = Lambda(channel_layer, arguments={'sigma': noise_sigma}, name='channel_layer')(e)

# Demodulation
d = Conv1D(filters=128, strides=1, kernel_size=1, padding="same", name='d_1')(y_h)
d = BatchNormalization(name='d_2')(d)
d = Activation('relu', name='d_3')(d)

d = LSTM(units=64,return_sequences='True',name='d_4')(d)
d = BatchNormalization(name='d_5')(d)
d = Activation('tanh', name='d_6')(d)

d = TimeDistributed(Dense(kmod))(d)
d = Reshape((L,n))(d)

# Decoder

d = Conv1D(filters=128, strides=1, kernel_size=1 ,padding='same', name='d_7',)(d)
d = BatchNormalization(name='d_8')(d)
d = Activation('relu', name='d_9')(d)

d = LSTM(units=128,return_sequences='True',name='d_10')(d)
d = BatchNormalization()(d)
d = Activation('relu')(d)

d = TimeDistributed(Dense(k),name='d_11')(d)
d = BatchNormalization(name='d_12')(d)
d = Reshape((L*k,1))(d)
model_output = Activation('sigmoid', name='d_13')(d)

# Build System Model
sys_model = Model(model_input, model_output)
encoder = Model(model_input, e)

# Print Model Architecture
sys_model.summary()

# Compile Model
sys_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# print('encoder output:', '\n', encoder.predict(vec_one_hot, batch_size=batch_size))



## ---TRAINING----

In [ ]:
print('starting train the NN...')
start = time.clock()

# TRAINING
mod_history = sys_model.fit(label_data, label_data,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_split=0.2, callbacks=[modelcheckpoint,reduce_lr])

end = time.clock()

print('The NN has trained ' + str(end - start) + ' s')

In [ ]:
# Plot the Training Loss and Validation Loss
hist_dict = mod_history.history

val_loss = hist_dict['val_loss']
loss = hist_dict['loss']
acc = hist_dict['accuracy']
# val_acc = hist_dict['val_acc']
#print('loss:',loss)
#print('val_loss:',val_loss)

epoch = np.arange(1, epochs + 1)

plt.semilogy(epoch,val_loss,label='val_loss')
plt.semilogy(epoch, loss, label='loss')

plt.legend(loc=0)
plt.grid('true')
plt.xlabel('epochs')
plt.ylabel('Binary cross-entropy loss')

plt.show()


## --- TESTING PARAMETERS ----

In [ ]:


batch_size = 500 # Number of messages used for test, each size = k*L
num_of_sym = 100000


# Initialize information Data 0/1
in_sym = np.random.randint(low=0, high=2, size=(num_of_sym, L,k))
#label_data = copy.copy(in_sym)
in_sym = np.reshape(in_sym,newshape=(num_of_sym,L*k,1))





## --- TESTING DATA GENERATION ---

## --- Simulation ---

In [ ]:
ber[i] = K.mean(K.cast(K.not_equal(test_data, K.round(pred_final_signal)),dtype='float32')).numpy()
bler = np.max((pred_final_signal.round()!=test_data),axis=1).mean()


### BER

In [ ]:


print('start simulation ...' + str(k) + '_' + str(L)+'_'+str(n))



'''
 --- DEFINE THE Neural Network(NN) ---
'''

# Eb_N0 in dB
for Eb_N0_dB in range(0,21):

    # Noise Sigma at this Eb
    noise_sigma = np.sqrt(1 / (2 * R * kmod*10 ** (Eb_N0_dB / 10)))

    # Define Encoder Layers (Transmitter)
    model_input = Input(batch_shape=(batch_size, L*k,1), name='input_bits')


    #Encoding
    e = Conv1D(filters=128, strides=1, kernel_size=1 ,padding='same', name='e_0',)(model_input)
    e = BatchNormalization()(e)
    e = Activation('relu')(e)

    e = LSTM(units=128,return_sequences='True',name='e_1')(e)
    e = BatchNormalization()(e)
    e = Activation('relu')(e)

    e = TimeDistributed(Dense(n))(e)
    e = Activation('linear')(e)
    e = Reshape((int(L*n/k_mod),k_mod))(e)

    #Modulation

    e = Conv1D(filters=128, strides=1, kernel_size=1, padding="same",)(e)
    e = BatchNormalization(name='e_2')(e)
    e = Activation('relu', name='e_3')(e)

    e = LSTM(units=64,return_sequences='True',name='e_4')(e)
    e = BatchNormalization(name='e_5')(e)
    e = Activation('tanh', name='e_6')(e)


    e = TimeDistributed(Dense(2),name='e_7')(e)
    e = Activation('linear', name='e_8')(e)

    e = Lambda(normalization, name='power_norm')(e)

    # AWGN channel
    y_h = Lambda(channel_layer, arguments={'sigma': noise_sigma}, name='channel_layer')(e)

    # Demodulation
    d = Conv1D(filters=128, strides=1, kernel_size=1, padding="same", name='d_1')(y_h)
    d = BatchNormalization(name='d_2')(d)
    d = Activation('relu', name='d_3')(d)

    d = LSTM(units=64,return_sequences='True',name='d_4')(d)
    d = BatchNormalization(name='d_5')(d)
    d = Activation('tanh', name='d_6')(d)

    d = TimeDistributed(Dense(kmod))(d)
    d = Reshape((L,n))(d)

    # Decoder

    d = Conv1D(filters=128, strides=1, kernel_size=1 ,padding='same', name='d_7',)(d)
    d = BatchNormalization(name='d_8')(d)
    d = Activation('relu', name='d_9')(d)

    d = LSTM(units=128,return_sequences='True',name='d_10')(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)

    d = TimeDistributed(Dense(k),name='d_11')(d)
    d = BatchNormalization(name='d_12')(d)
    d = Reshape((L*k,1))(d)
    model_output = Activation('sigmoid', name='d_13')(d)
        
    
    # Build the model
    model = Model(inputs=model_input, outputs=model_output)

    encoder = Model(model_input,e)
    # Load Weights from the trained NN
    model.load_weights('./' + 'PAPER2' + str(k) + '_' + str(L) + '_' + str(n) + '_' + str(train_Eb_dB) + 'dB' + ' ' + 'AWGN' + '.h5')

    '''
    RUN THE NN
    '''

    # RUN Through the Model and get output
    decoder_output = model.predict(label_data, batch_size=batch_size)
    #encoder_output = encoder.predict(vec_one_hot, batch_size=batch_size)

    '''
     --- CALULATE BLER ---
    '''

    # Decode One-Hot vector

    tmp = (decoder_output.round()!=label_data).reshape(label_data.shape[0],label_data.shape[1])
    error_rate = np.mean(tmp)
    print('Eb/N0 = ', Eb_N0_dB,'BER = ', error_rate)


    # Store The Results
    Vec_Eb_N0.append(Eb_N0_dB)
    Bit_error_rate.append(error_rate)





## BLER

In [ ]:

print('start simulation ...' + str(k) + '_' + str(L)+'_'+str(n))



'''
 --- DEFINE THE Neural Network(NN) ---
'''

# Eb_N0 in dB
for Eb_N0_dB in range(0,21):

    # Noise Sigma at this Eb
    noise_sigma = np.sqrt(1 / (2 * R * 10 ** (Eb_N0_dB / 10)))

    model_input = Input(batch_shape=(None, L*k,1), name='input_bits')

    e = Conv1D(filters=128, strides=k, kernel_size=k, name='e_1',)(model_input)
    e = BatchNormalization(name='e_2')(e)
    e = Activation('relu', name='e_3')(e)

    e = LSTM(units=32,return_sequences='True',name='e_4')(e)
    e = BatchNormalization(name='e_5')(e)
    e = Activation('tanh', name='e_6')(e)

    e = TimeDistributed(Dense(2*n),name='e_7')(e)
    e = Activation('linear', name='e_8')(e)
    e = Reshape((e.shape[1]*n,2),name='e_9')(e)

    e = Lambda(normalization, name='power_norm')(e)

    # AWGN channel
    y_h = Lambda(channel_layer, arguments={'sigma': noise_sigma}, name='channel_layer')(e)

    # Define Decoder Layers (Receiver)
    d = LSTM(units=128,return_sequences='True',name='d_1')(y_h)
    d = BatchNormalization(name='d_2')(d)
    d = Activation('tanh', name='d_3')(d)

    d = LSTM(units=64,return_sequences='True',name='d_4')(d)
    d = BatchNormalization(name='d_5')(d)
    d = Activation('tanh', name='d_6')(d)

    d = Conv1D(filters=64, strides=1, kernel_size=1, name='d_7')(d)
    d = BatchNormalization(name='d_8')(d)
    d = Activation('relu', name='d_9')(d)

    d = TimeDistributed(Dense(n),name='d_10')(d)
    d = BatchNormalization(name='d_11')(d)
    d = Reshape((L*k,1),name='d_12')(d)
    model_output = Activation('sigmoid', name='d_13')(d)
    
    
    # Build the model
    model = Model(inputs=model_input, outputs=model_output)

    encoder = Model(model_input,e)
    # Load Weights from the trained NN
    model.load_weights('./' + 'PAPER2' + str(k) + '_' + str(L) + '_' + str(n) + '_' + str(train_Eb_dB) + 'dB' + ' ' + 'AWGN' + '.h5',
                       by_name=False)

    '''
    RUN THE NN
    '''

    # RUN Through the Model and get output
    decoder_output = model.predict(label_data, batch_size=batch_size)
    #encoder_output = encoder.predict(vec_one_hot, batch_size=batch_size)

    '''
     --- CALULATE BLER ---
    '''

    # Decode One-Hot vector

    resh= (decoder_output.round()!=label_data).reshape(label_data.shape[0],label_data.shape[1])
    tmp = np.max(resh,axis=1)
    error_rate = np.mean(tmp)
    print('Eb/N0 = ', Eb_N0_dB,'BLER = ', error_rate)


    # Store The Results
    Vec_Eb_N0.append(Eb_N0_dB)
    Block_error_rate.append(error_rate)

## --- PLOTTING ---

### BER

In [ ]:

# Print BER
# print(Bit_error_rate)

#print(Vec_Eb_N0, '\n', Bit_error_rate)

#with open('BLER_model_LBC_'+str(k)+'_'+str(n)+'_'+str(L)+'_AWGN'+'.txt', 'w') as f:
#    print(Vec_Eb_N0, '\n', Bit_error_rate, file=f)
#f.closed

# Plot BER Figure
plt.semilogy(Vec_Eb_N0, Bit_error_rate, color='red')
label = ['k= ' +str(k) + ',' + 'L= '+ str(L)]
plt.legend(label, loc=0)
plt.xlabel('Eb/N0')
plt.ylabel('BER')
plt.title('k= ' + str(k) + ', ' + 'n= ' +str(n)+', ' + 'L= ' +str(L))
plt.grid('true')
plt.show()